# Demo notebook for slide 3.3

In [1]:
%matplotlib widget

In [2]:
import bmcs_utils

In [3]:
from bmcs_matmod.slide.slide_explorer import SlideExplorer
import bmcs_utils.api as bu

In [4]:
se = SlideExplorer()

In [5]:
sm = se.slide_model.C_codegen()

generated code_file /home/rch/sympy_codegen/SLIDE33_3D


Parameters for elementary convergence studies - no physical relevance

Parameters rendering a stress-strain response comparable with a `MARS` model - example concrete

# Debugging LDM implementation

In [6]:
material_params = dict(
    E_s=1, gamma_s=1, K_s=1, S_s=0.5, c_s=1, bartau=1, 
    E_w=1, S_w=0.5, c_w = 1, m = 0.05, f_t=1, f_c=20, f_c0=10, eta=1
)

Critical step size - step smaller than the elastic range in tension.

In [7]:
def get_step_cr(**kw):
    f_t = kw['f_t']
    E_w = kw['E_w']
    return f_t/E_w
d_w = get_step_cr(**material_params)
w_max = 10
n_steps = int(w_max / d_w)
n_steps

10

In [10]:
%%capture
se = SlideExplorer(n_steps=200, k_max=50)
se.slide_model.trait_set(**material_params)
se.trait_set(s_x_1 = 5, s_y_1 = 0.1, w_1 = 0);
se.run()
se.trait_set(s_x_1 = 2, s_y_1 = 1, w_1 = 0);
se.run()
se.trait_set(s_x_1 = 2, s_y_1 = 1, w_1 = 10);
se.run()

In [11]:
se.energy_dissipation.show_iter = False
se.inel_state_evolution.t_max=3.0
bu.InteractiveWindow([se.inel_state_evolution, se, se.energy_dissipation, 
                      se.slide_model], figsize=(9,4), tight_layout=False).interact()

In [12]:
%%capture
import numpy as np
eta_range = np.linspace(0,1,4)
se_T_list = []
se_N_list = []
for eta in eta_range:
    material_params['eta'] = eta
    se1 = SlideExplorer(n_steps=200, k_max=50)
    se1.slide_model.trait_set(**material_params)
    se1.trait_set(s_x_1 = 5, s_y_1 = 0, w_1 = 0);
    se1.run()
    se_T_list.append(se1)
    se2 = SlideExplorer(n_steps=200, k_max=50)
    se2.slide_model.trait_set(**material_params)
    se2.trait_set(s_x_1 = 0, s_y_1 = 0, w_1 = 6);
    se2.run()
    se_N_list.append(se2)

In [13]:
import matplotlib.pylab as plt
_, (ax_T, ax_N) = plt.subplots(1,2)
for se_T, se_N in zip(se_T_list, se_N_list):
    se_T.inel_state_evolution.plot_omega_NT(ax_T)
    se_N.inel_state_evolution.plot_omega_NT(ax_N)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Debugging support

You can access the values of the state variables and of the the associated 
thermodynamic forces by accessing the `Sig_t` and `Eps_t` arrays.
Each row represents the time step and each column a state variable.
The order of state variables corresponds to the order in the symbolic 
object `se.slide_model.symb`

In [8]:
se.Eps_t[-1,:], se.Sig_t[-1,:]

(array([1.51734057, 0.85063411, 8.99999555, 0.39270458, 0.48264735,
        0.14935572, 0.93830544, 0.93613676]),
 array([0.02977746, 0.00921506, 0.06386352, 0.39270458, 0.48264735,
        0.14935572, 0.12763515, 0.50000445]))

In [9]:
se.Sig_arr[-1,:]

array([0.02977746, 0.00921506, 0.06386352, 0.39270458, 0.48264735,
       0.14935572, 0.12763515, 0.50000445])